# SCPI ramps and Thales EIS without turning off the potentiostat

All other examples are needed as a precognition for this example. Especially the [Ramps.ipynb](https://github.com/Zahner-elektrik/Zahner-Remote-Python/blob/main/Examples/Ramps/Ramps.ipynb) and [ImpedanceMultiCellCycle.ipynb](https://github.com/Zahner-elektrik/Thales-Remote-Python/blob/main/Examples/ImpedanceMultiCellCycle/ImpedanceMultiCellCycle.ipynb) are important and must be understand before.

In this example, different DC currents are set with ramps. The ramp is followed by a polarization phase, which is followed by an impedance spectrum at this current setting. The DC current settings can be parameterized variably.

**This notebook cannot be executed and has been created only for documentation and explanation of the source code, because Jupyter does not support loops over multiple cells.**

In [ ]:
from thales_remote.epc_scpi_handler import EpcScpiHandlerFactory,EpcScpiHandler
from thales_remote.script_wrapper import PotentiostatMode
from zahner_potentiostat.scpi_control.datahandler import DataManager
from zahner_potentiostat.display.onlinedisplay import OnlineDisplay
from zahner_potentiostat.scpi_control.datareceiver import TrackTypes

# Definition of utiltiy functions

A class is defined, which contains the parameters for the measurement. This simplifies the process and makes it easier to change the parameters.

Two functions are also programmed to show that the same parameters are measured before and after the operating mode change.

In [ ]:
class TargetCurrents:
    def __init__(self, dc, amplitude, scanrate):
        self.dc = dc
        self.amplitude = amplitude
        self.scanrate = scanrate
        return

def measure_UI_EPC(deviceHandler):
    for _ in range(3):
        print(f"EPC-Potential:\t{deviceHandler.sharedZenniumInterface.getPotential():>10.6f} V")
        print(f"EPC-Current:\t{deviceHandler.sharedZenniumInterface.getCurrent():>10.3e} A")
    return

def measure_UI_SCPI(deviceHandler):
    for _ in range(3):
        print(f"SCPI-Potential:\t{deviceHandler.scpiInterface.getPotential():>10.6f} V")
        print(f"SCPI-Current:\t{deviceHandler.scpiInterface.getCurrent():>10.3e} A")
    return

# Initialization

In the example [ImpedanceMultiCellCycle.ipynb](https://github.com/Zahner-elektrik/Thales-Remote-Python/blob/main/Examples/ImpedanceMultiCellCycle/ImpedanceMultiCellCycle.ipynb) the initialization is explained step by step.

For switching between EPC and SCPI operation, it is important that both instruments have warmed up for 30 minutes and that the calibration routine has been performed in both operating modes. This ensures that the DC differences are minimized.

In [ ]:
if __name__ == "__main__": 
    startCurrent = 0.0
    handlerFactory = EpcScpiHandlerFactory("192.168.2.94")
    deviceHandler = handlerFactory.createEpcScpiHandler(epcChannel=1, serialNumber=33021)

After all devices have been warmed up for 30 minutes, the first step is to calibrate for SCPI mode after initialization.

Calibration is also performed for the EPC mode.

In [ ]:
    deviceHandler.scpiInterface.calibrateOffsets()
    
    deviceHandler.acquireSharedZennium(blocking = True)
    deviceHandler.switchToEPC()
    
    deviceHandler.sharedZenniumInterface.calibrateOffsets()

After everything has been calibrated, the potentiostat can be switched on.

In [ ]:
    deviceHandler.sharedZenniumInterface.setPotentiostatMode(PotentiostatMode.POTMODE_GALVANOSTATIC)
    deviceHandler.sharedZenniumInterface.setCurrent(startCurrent)
    deviceHandler.sharedZenniumInterface.enablePotentiostat()

After switching on in EPC mode, current and voltage are measured in EPC mode and output on the console.

Then [switchToSCPIAndReleaseSharedZennium(keepPotentiostatState = True)](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler.switchToSCPIAndReleaseSharedZennium) switches to SCPI mode without switching off the potentiostat. With this function also the Zennium is released that it could be used by other parallel channels as in the other example.

A current and voltage measurement on the SCPI side follows.

In [ ]:
    measure_UI_EPC(deviceHandler)
    deviceHandler.switchToSCPIAndReleaseSharedZennium(keepPotentiostatState = True)
    measure_UI_SCPI(deviceHandler)

# EIS and ramps with different currents

The class definied in the previous is now used to define the steps for the measurement. For each step the target DC current, the EIS amplitude and the scanrate to set the DC current is defined.

In [ ]:
    measurementSettings = [
        TargetCurrents(dc = 1, amplitude = 0.1, scanrate=0.1),
        TargetCurrents(dc = 2, amplitude = 0.1, scanrate=0.1),
        TargetCurrents(dc = 4, amplitude = 0.2, scanrate=0.5)
    ]

The next step is to iterate over the specified steps with a for loop.

In each iteration, exactly the same measurement is performed, but with different parameters from the objects.

In [ ]:
    for setting in measurementSettings:

The ramp starts at the actual current value therfore the actual current is read and set as value.

An online display for the SCPI measurements is also configured and started.

In [ ]:
        deviceHandler.scpiInterface.setCurrentValue(deviceHandler.scpiInterface.getCurrent())

        configuration = {
        "figureTitle":f"Online Display Targetcurrent: {setting.dc} Scanrate: {setting.scanrate}",
        "xAxisLabel":"Time",
        "xAxisUnit":"s",
        "xTrackName":TrackTypes.TIME.toString(),
        "yAxis":
            [{"label": "Voltage", "unit": "V", "trackName":TrackTypes.VOLTAGE.toString()},
             {"label": "Current", "unit": "A", "trackName":TrackTypes.CURRENT.toString()}]
        }
        onlineDisplay = OnlineDisplay(deviceHandler.scpiInterface.getDataReceiver(), displayConfiguration=configuration)
        

The values from the current object of the iterator are now entered as parameters for the ramp. Then the ramp is executed.

In [ ]:
        deviceHandler.scpiInterface.setCurrentParameter(setting.dc)
        deviceHandler.scpiInterface.setScanRateParameter(setting.scanrate)
        deviceHandler.scpiInterface.measureRampValueInScanRate()

Followed by the ramp, the DC current is held for at least 5 seconds or until the common Zennium is available.

In this example, no other devices use the zennium, so it is always available.

In [ ]:
        deviceHandler.scpiInterface.setMaximumTimeParameter(5)
        deviceHandler.scpiInterface.measurePolarization()
        while deviceHandler.acquireSharedZennium(blocking = False) == False:
            deviceHandler.scpiInterface.measurePolarization()

After the Zennium is acquired, the measurement data is saved with the target current and scan rate as the filename.

In [ ]:
        dataManager = DataManager(deviceHandler.scpiInterface.getDataReceiver())
        dataManager.saveDataAsText(f"ramp_to{setting.dc}a_{setting.scanrate}apers.txt")
        
        onlineDisplay.close()
        del onlineDisplay
        del dataManager

To switch to EPC control with activated potentiostat for EIS measurement, the method [switchToEPC()](https://doc.zahner.de/thales_remote/epc_scpi_handler.html#thales_remote.epc_scpi_handler.EpcScpiHandler.switchToSCPI) with the parameter *keepPotentiostatState = True* is used.

When switching to EPC, it may be the case that the potentiostat is switched off for about 50 ms.

Before and after the operation mode switch current and voltage are displayed on the console.

In [ ]:
        measure_UI_SCPI(deviceHandler)
        deviceHandler.switchToEPC(keepPotentiostatState = True)
        measure_UI_EPC(deviceHandler)

Afterwards an EIS measurement is setup and executed.

The measurement is saved with DC current and amplitude in the filename and is stored by Thales on the hard disk.

In [ ]:
        deviceHandler.sharedZenniumInterface.setEISNaming("individual")
        deviceHandler.sharedZenniumInterface.setEISOutputPath(r"C:\THALES\temp")
        deviceHandler.sharedZenniumInterface.setEISOutputFileName(f"{setting.dc}adc_{setting.amplitude}aac".replace(".",""))

        deviceHandler.sharedZenniumInterface.setPotentiostatMode(PotentiostatMode.POTMODE_GALVANOSTATIC)
        deviceHandler.sharedZenniumInterface.setPotential(setting.dc)
        deviceHandler.sharedZenniumInterface.setAmplitude(setting.amplitude)
        
        deviceHandler.sharedZenniumInterface.setLowerFrequencyLimit(100)
        deviceHandler.sharedZenniumInterface.setStartFrequency(250)
        deviceHandler.sharedZenniumInterface.setUpperFrequencyLimit(500)
        deviceHandler.sharedZenniumInterface.setLowerNumberOfPeriods(5)
        deviceHandler.sharedZenniumInterface.setLowerStepsPerDecade(10)
        deviceHandler.sharedZenniumInterface.setUpperNumberOfPeriods(20)
        deviceHandler.sharedZenniumInterface.setUpperStepsPerDecade(10)
        deviceHandler.sharedZenniumInterface.setScanDirection("startToMax")
        deviceHandler.sharedZenniumInterface.setScanStrategy("single")
        
        deviceHandler.sharedZenniumInterface.enablePotentiostat()
        deviceHandler.sharedZenniumInterface.measureEIS()
        deviceHandler.sharedZenniumInterface.setAmplitude(0)

At the end, the system switches back to SCPI mode and the potentiostat is switched off.

In [ ]:
        measure_UI_EPC(deviceHandler)
        deviceHandler.switchToSCPIAndReleaseSharedZennium(keepPotentiostatState = True)
        measure_UI_SCPI(deviceHandler)
    
    deviceHandler.scpiInterface.setPotentiostatEnabled(False)
    handlerFactory.closeAll()
    print("finish")